In [1]:
import pandas as pd
import numpy as np
import collections
import re
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import log_loss, roc_auc_score
import warnings
import torch.nn as nn
from tqdm import tqdm
import random
import gensim
from torchcontrib.optim import SWA
import os
from torch.utils import data
from torch import nn
import torch.nn.functional as F
from torch.optim import *
torch.set_printoptions(edgeitems=768)
warnings.filterwarnings("ignore")
np.set_printoptions(threshold=np.inf)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# 设置基本参数
MAX_LEN = 100
BATCH_SIZE = 16
SEED = 9797
NAME = 'capsuleNet'
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if DEVICE=='cuda':
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
DEVICE

'cuda'

In [2]:
train_data = pd.read_csv('../data/track1_round1_train_20210222.csv',header=None)
test_data = pd.read_csv('../data/track1_round1_testB.csv',header=None) 
train_data.columns=['report_ID','description','label']
test_data.columns=['report_ID','description']

temp=[i[:-1] for i in train_data['report_ID'].values]
train_data['report_ID']=temp
temp=[i[:-1] for i in test_data['report_ID'].values]
test_data['report_ID']=temp

temp=[i.strip('|').strip() for i in train_data['description'].values]
train_data['description']=temp
temp=[i.strip('|').strip() for i in test_data['description'].values]
test_data['description']=temp

temp_label=[i.strip('|').strip() for i in train_data['label'].values]
train_data['label']=temp_label
train_data

,report_ID,description,label
0,0,623 328 538 382 399 400 478 842 698 137 492 26...,2
1,1,48 328 538 382 809 623 434 355 382 382 363 145...,
2,2,623 656 293 851 636 842 698 493 338 266 369 69...,15
3,3,48 328 380 259 439 107 380 265 172 470 290 693...,
4,4,623 328 399 698 493 338 266 14 177 415 511 647...,16
...,...,...,...
9995,9995,290 380 247 263 48 328 697 582 91 400 478 842 ...,0 7 15
9996,9996,852 611 501 582 177 230 294 39 363 180 519 421...,10
9997,9997,852 328 290 380 256 544 636 90 735 374 698 116...,
9998,9998,852 328 305 461 382 697 259 779 59 261 589 693...,16


In [3]:
all_sentences = pd.concat([train_data['description'],test_data['description']]).reset_index(drop=True)
all_sentences.drop_duplicates().reset_index(drop=True, inplace=True)
all_sentences = all_sentences.apply(lambda x:x.split(' ')).tolist()
if not os.path.exists('../embedding/w2v.model'): 
    w2v_model = gensim.models.word2vec.Word2Vec(all_sentences, sg=1, size=300,window=7,min_count=1,negative=3,sample=0.001,hs=1,seed=452)
    w2v_model.save('../embedding/w2v.model')
else:
    w2v_model = gensim.models.word2vec.Word2Vec.load("../embedding/w2v.model")
    
if not os.path.exists('../embedding/fasttext.model'): 
    fasttext_model = gensim.models.FastText(all_sentences, seed=452, size=100, min_count=1, iter=20, window=2)
    fasttext_model.save('../embedding/fasttext.model')
else:
    fasttext_model = gensim.models.word2vec.Word2Vec.load("../embedding/fasttext.model")

In [4]:
train_dataset = []
for i in tqdm(range(len(train_data))):
    train_dict = {}
    train_dict['report_ID'] = train_data.loc[i, 'report_ID']
    train_dict['description'] = train_data.loc[i, 'description']
    train_dict['label'] = train_data.loc[i, 'label']
    train_dataset.append(train_dict)
test_dataset = []
for i in tqdm(range(len(test_data))):
    test_dict = {}
    test_dict['report_ID'] = test_data.loc[i, 'report_ID']
    test_dict['description'] = test_data.loc[i, 'description']
    test_dict['label'] = ''
    test_dataset.append(test_dict)

In [5]:
class DataSet(data.Dataset):
    def __init__(self, data, mode='train'):
        self.data = data
        self.mode = mode
        self.dataset = self.get_data(self.data,self.mode)
        
    def get_data(self, data, mode):
        dataset = []
        global s
        for data_li in tqdm(data):
            description = data_li['description'].split(' ')
            description = [w2v_model.wv.vocab[s].index+1 if s in w2v_model.wv.vocab else 0 for s in description]
            if len(description) < MAX_LEN:
                description += [0] * (MAX_LEN - len(description))
            else:
                description = description[:MAX_LEN]
            label = self.get_dumm(data_li['label'])
            dataset_dict = {'description':description, 'label':label}
            dataset.append(dataset_dict)
        return dataset
    
    def get_dumm(self,s):
        re = [0] * 17
        if s == '':
            return re
        else:
            tmp = [int(i) for i in s.split(' ')]
            for i in tmp:
                re[i] = 1
        return re
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        data = self.dataset[idx]
        description = torch.tensor(data['description'])
        if self.mode == 'test':
            return description
        else:
            label = torch.tensor(data['label'])
            return description, label

def get_dataloader(dataset, mode):
    torchdata = DataSet(dataset, mode=mode)
    if mode == 'train':
        dataloader = torch.utils.data.DataLoader(torchdata, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, drop_last=True)
    elif mode == 'test':
        dataloader = torch.utils.data.DataLoader(torchdata, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, drop_last=False)
    elif mode == 'valid':
        dataloader = torch.utils.data.DataLoader(torchdata, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, drop_last=False)
    return dataloader, torchdata

train_dataloader, train_torchdata = get_dataloader(train_dataset, mode='train')
test_dataloader, test_torchdata = get_dataloader(test_dataset, mode='test')

In [6]:
class CyclicLR(object):
    def __init__(self, optimizer, base_lr=1e-3, max_lr=6e-3,
                 step_size=2000, mode='triangular', gamma=1.,
                 scale_fn=None, scale_mode='cycle', last_batch_iteration=-1):

        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(
                type(optimizer).__name__))
        self.optimizer = optimizer

        if isinstance(base_lr, list) or isinstance(base_lr, tuple):
            if len(base_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} base_lr, got {}".format(
                    len(optimizer.param_groups), len(base_lr)))
            self.base_lrs = list(base_lr)
        else:
            self.base_lrs = [base_lr] * len(optimizer.param_groups)

        if isinstance(max_lr, list) or isinstance(max_lr, tuple):
            if len(max_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} max_lr, got {}".format(
                    len(optimizer.param_groups), len(max_lr)))
            self.max_lrs = list(max_lr)
        else:
            self.max_lrs = [max_lr] * len(optimizer.param_groups)

        self.step_size = step_size

        if mode not in ['triangular', 'triangular2', 'exp_range'] \
                and scale_fn is None:
            raise ValueError('mode is invalid and scale_fn is None')

        self.mode = mode
        self.gamma = gamma

        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = self._triangular_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = self._triangular2_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = self._exp_range_scale_fn
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode

        self.batch_step(last_batch_iteration + 1)
        self.last_batch_iteration = last_batch_iteration

    def batch_step(self, batch_iteration=None):
        if batch_iteration is None:
            batch_iteration = self.last_batch_iteration + 1
        self.last_batch_iteration = batch_iteration
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

    def _triangular_scale_fn(self, x):
        return 1.

    def _triangular2_scale_fn(self, x):
        return 1 / (2. ** (x - 1))

    def _exp_range_scale_fn(self, x):
        return self.gamma**(x)

    def get_lr(self):
        step_size = float(self.step_size)
        cycle = np.floor(1 + self.last_batch_iteration / (2 * step_size))
        x = np.abs(self.last_batch_iteration / step_size - 2 * cycle + 1)

        lrs = []
        param_lrs = zip(self.optimizer.param_groups, self.base_lrs, self.max_lrs)
        for param_group, base_lr, max_lr in param_lrs:
            base_height = (max_lr - base_lr) * np.maximum(0, (1 - x))
            if self.scale_mode == 'cycle':
                lr = base_lr + base_height * self.scale_fn(cycle)
            else:
                lr = base_lr + base_height * self.scale_fn(self.last_batch_iteration)
            lrs.append(lr)
        return lrs

In [7]:
Num_capsule = 5
Dim_capsule = 5
class Caps_Layer(nn.Module):
    def __init__(self, input_dim_capsule, num_capsule=Num_capsule, dim_capsule=Dim_capsule, \
                 routings=4, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Caps_Layer, self).__init__(**kwargs)
        self.T_epsilon = 1e-7
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = 4
        self.kernel_size = kernel_size  # 暂时没用到
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = self.squash
        else:
            self.activation = nn.ReLU(inplace=True)

        if self.share_weights:
            self.W = nn.Parameter(
                nn.init.xavier_normal_(torch.empty(1, input_dim_capsule, self.num_capsule * self.dim_capsule)))
        else:
            self.W = nn.Parameter(
                torch.randn(BATCH_SIZE, input_dim_capsule, self.num_capsule * self.dim_capsule))  # 64即batch_size

    def forward(self, x):

        if self.share_weights:
            u_hat_vecs = torch.matmul(x, self.W)
        else:
            print('add later')

        batch_size = x.size(0)
        input_num_capsule = x.size(1)
        u_hat_vecs = u_hat_vecs.view((batch_size, input_num_capsule,
                                      self.num_capsule, self.dim_capsule))
        u_hat_vecs = u_hat_vecs.permute(0, 2, 1, 3)  # 转成(batch_size,num_capsule,input_num_capsule,dim_capsule)
        b = torch.zeros_like(u_hat_vecs[:, :, :, 0])  # (batch_size,num_capsule,input_num_capsule)

        for i in range(self.routings):
            b = b.permute(0, 2, 1)
            c = F.softmax(b, dim=2)
            c = c.permute(0, 2, 1)
            b = b.permute(0, 2, 1)
            outputs = self.activation(torch.einsum('bij,bijk->bik', (c, u_hat_vecs)))  # batch matrix multiplication
            # outputs shape (batch_size, num_capsule, dim_capsule)
            if i < self.routings - 1:
                b = torch.einsum('bik,bijk->bij', (outputs, u_hat_vecs))  # batch matrix multiplication
        return outputs  # (batch_size, num_capsule, dim_capsule)

    # text version of squash, slight different from original one
    def squash(self, x, axis=-1):
        s_squared_norm = (x ** 2).sum(axis, keepdim=True)
        scale = torch.sqrt(s_squared_norm + self.T_epsilon)
        return x / scale
    
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1)
        nn.init.xavier_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / torch.sum(a, 1, keepdim=True) + 1e-10

        weighted_input = x * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)
    
class NeuralNet(nn.Module):
    def __init__(self,vocab_size,embedding_dim,embeddings=None):
        super(NeuralNet, self).__init__()
        self.num_classes = 17
        fc_layer = 256
        hidden_size = 128
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        if embeddings:
            w2v_model = gensim.models.word2vec.Word2Vec.load("../embedding/w2v.model").wv
            fasttext_model = gensim.models.word2vec.Word2Vec.load("../embedding/fasttext.model").wv
            w2v_embed_matrix = w2v_model.vectors
            fasttext_embed_matrix = fasttext_model.vectors
#             embed_matrix = w2v_embed_matrix         
            embed_matrix = np.concatenate([w2v_embed_matrix, fasttext_embed_matrix], axis=1)
            oov_embed = np.zeros((1, embed_matrix.shape[1]))
            embed_matrix = torch.from_numpy(np.vstack((oov_embed,embed_matrix)))
            self.embedding.weight.data.copy_(embed_matrix)
            self.embedding.weight.requires_grad = False
        
        self.embedding_dropout = nn.Dropout2d(0.1)
        self.lstm = nn.GRU(embedding_dim, hidden_size,2, bidirectional=True, batch_first=True)
        self.gru = nn.GRU(hidden_size * 2, hidden_size,2, bidirectional=True, batch_first=True)
        self.tdbn = nn.BatchNorm2d(1)
        self.lstm_attention = Attention(hidden_size * 2, MAX_LEN)
        self.gru_attention = Attention(hidden_size * 2, MAX_LEN)
        self.bn = nn.BatchNorm1d(fc_layer)
        self.linear = nn.Linear(hidden_size*8+1, fc_layer)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.output = nn.Linear(fc_layer, self.num_classes)
        self.lincaps = nn.Linear(Num_capsule * Dim_capsule, 1)
        self.caps_layer = Caps_Layer(hidden_size*2)
    def forward(self, x, label=None):
        
#         Capsule(num_capsule=10, dim_capsule=10, routings=4, share_weights=True)(x)
        h_embedding = self.embedding(x)
        h_embedding = torch.squeeze(
            self.embedding_dropout(torch.unsqueeze(h_embedding, 0)))
        h_embedding = self.tdbn(h_embedding.unsqueeze(1)).squeeze(1)
        h_lstm, _ = self.lstm(h_embedding)
        h_gru, _ = self.gru(h_lstm)

        ##Capsule Layer        
        content3 = self.caps_layer(h_gru)
        content3 = self.dropout(content3)
        batch_size = content3.size(0)
        content3 = content3.view(batch_size, -1)
        content3 = self.relu(self.lincaps(content3))

        ##Attention Layer
        h_lstm_atten = self.lstm_attention(h_lstm)
        h_gru_atten = self.gru_attention(h_gru)
        
        # global average pooling
        avg_pool = torch.mean(h_gru, 1)
        # global max pooling
        max_pool, _ = torch.max(h_gru, 1)
        
        conc = torch.cat((h_lstm_atten, h_gru_atten,content3, avg_pool, max_pool), 1)
        conc = self.relu(self.linear(conc))
        conc = self.bn(conc)
        out = self.dropout(self.output(conc))
        if label is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(out.view(-1,self.num_classes).float(), label.view(-1,self.num_classes).float())
            return loss
        else:
            return out

In [8]:
def metric_mlogloss(label,pred):
    score = 0
    for i in range(len(pred)):
        for j in range(17):
            if pred[i][j] == 0:
                pred[i][j] +=1e-10
            elif pred[i][j] == 1:
                pred[i][j] -=1e-10
            score += label[i][j]*np.log(pred[i][j])+(1-label[i][j])*np.log(1-pred[i][j])
    score /= (len(pred)*17*(-1))
    return 1-score

def validation_funtion(model, valid_dataloader, valid_torchdata, mode='valid'):
    model.eval()
    pred_list = []
    labels_list = []
    if mode == 'valid':
        for i, (description, label) in enumerate(tqdm(valid_dataloader)):
            output = model(description.to(DEVICE))
            pred_list += output.sigmoid().detach().cpu().numpy().tolist()
            labels_list += label.detach().cpu().numpy().tolist()
        auc = roc_auc_score(labels_list,pred_list, multi_class='ovo')
        logloss = log_loss(labels_list, pred_list)
        mlogloss = metric_mlogloss(labels_list, pred_list)
        return mlogloss, auc, logloss
    else:
        for i, (description) in enumerate(tqdm(valid_dataloader)):
            output = model(description.to(DEVICE))
            pred_list += output.sigmoid().detach().cpu().numpy().tolist()
        return pred_list
    
                            
def train(model, train_dataloader, valid_dataloader, valid_torchdata, epochs, early_stop=None):
    global logger
#     ema = EMA(model, 0.999)
#     ema.register()
    param_optimizer = list(model.named_parameters())
    embed_pa = ['embedding.weight']
    optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer if not any(nd in n for nd in embed_pa)]},
                                    {'params': model.embedding.parameters(), 'lr': 5e-5}]
    optimizer = AdamW(optimizer_grouped_parameters, lr=1e-3, amsgrad=True, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=3, T_mult=2, eta_min=1e-5, last_epoch=-1)
#     scheduler = CyclicLR(optimizer, base_lr=1e-3, max_lr=3e-3,
#                step_size=30, mode='exp_range',
#                gamma=0.99994)
#     opt = SWA(optimizer, swa_start=100, swa_freq=5, swa_lr=1e-4)
    total_loss = []
    train_loss = []
    best_mlogloss = -np.inf
    best_auc = -np.inf
    best_loss = np.inf
    no_improve = 0
    for epoch in range(epochs):
        model.train()
#         fgm = FGM(model)
        bar = tqdm(train_dataloader)
        for i, (description, label) in enumerate(bar):
            optimizer.zero_grad()
            output = model(description.to(DEVICE), label.to(DEVICE))
            loss = output
            loss.backward()
            train_loss.append(loss.item())
            
#             fgm.attack()
#             loss_adv = model(describe.to(DEVICE), label.to(DEVICE))
#             loss_ad = loss_adv
#             loss_ad.backward()
#             fgm.restore()
            
            scheduler.step(epochs + i / len(train_dataloader))
#             scheduler.batch_step()
            optimizer.step()
#             ema.update()
            bar.set_postfix(tloss=np.array(train_loss).mean())
#         opt.swap_swa_sgd()
#         ema.apply_shadow()
        mlogloss, auc, logloss = validation_funtion(model, valid_dataloader, valid_torchdata, 'valid')
#         ema.restore()
        print('train_loss: {:.5f}, mlogloss: {:.5f}, auc: {:.5f}, log_loss: {:.5f}\n'.format(train_loss[-1],mlogloss,auc,logloss))
        logger.info('Epoch:[{}]\t mlogloss={:.5f}\t auc={:.5f}\t log_loss={:.5f}\t'.format(epoch,mlogloss,auc,logloss))
        global model_num
        if early_stop:
            if mlogloss > best_mlogloss:
                best_mlogloss = mlogloss
                best_auc = auc
                best_loss = train_loss[-1]
#                 ema.apply_shadow()
                torch.save(model.state_dict(), '{}_model_{}.bin'.format(NAME, model_num))
#                 ema.restore()
            else:
                no_improve += 1
            if no_improve == early_stop:
                model_num += 1
                break
            if epoch == epochs-1:
                model_num += 1
        else:
            if epoch >= epochs-1:
                torch.save(model.state_dict(), '{}_model_{}.bin'.format(NAME, model_num))
                model_num += 1
    return best_mlogloss, best_auc, best_loss

In [9]:
import logging
def get_logger(filename, verbosity=1, name=None):
    level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
    formatter = logging.Formatter(
        "[%(asctime)s][%(filename)s][line:%(lineno)d][%(levelname)s] %(message)s"
    )
    logger = logging.getLogger(name)
    logger.setLevel(level_dict[verbosity])
    fh = logging.FileHandler(filename, "w")
    fh.setFormatter(formatter)
    logger.addHandler(fh)
    sh = logging.StreamHandler()
    sh.setFormatter(formatter)
    logger.addHandler(sh)
    logger.removeHandler(sh)
    return logger

In [10]:
FOLD = 10
kf = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=SEED)
model_num = 1
test_preds_total = collections.defaultdict(list)
logger = get_logger('{}.log'.format(NAME))
best_mlogloss = []
best_auc = []
best_loss = []
for i, (train_index, test_index) in enumerate(kf.split(np.arange(train_data.shape[0]), train_data.label.values)):
    print(str(i+1), '-'*50)
    tra = [train_dataset[index] for index in train_index]
    val = [train_dataset[index] for index in test_index]
    print(len(tra))
    print(len(val))
    train_dataloader, train_torchdata = get_dataloader(tra, mode='train')
    valid_dataloader, valid_torchdata = get_dataloader(val, mode='valid')
    model = NeuralNet(w2v_model.wv.vectors.shape[0]+1,w2v_model.wv.vectors.shape[1]+fasttext_model.wv.vectors.shape[1],embeddings=True)
    model.to(DEVICE)
    mlogloss,auc,loss = train(model,train_dataloader,
                    valid_dataloader,
                    valid_torchdata,
                    epochs=100,
                    early_stop=5)
    torch.cuda.empty_cache()
    best_mlogloss.append(mlogloss)
    best_auc.append(auc)
    best_loss.append(loss)
for i in range(FOLD):
    print('- 第{}折中，best mlogloss: {}   best auc: {}   best loss: {}'.format(i+1, best_mlogloss[i], best_auc[i], best_loss[i]))

- 第1折中，best mlogloss: 0.9754145609566004   best auc: 0.9989513662197707   best loss: 0.370415061712265
- 第2折中，best mlogloss: 0.9775772409438741   best auc: 0.9973311857745906   best loss: 0.3379441797733307
- 第3折中，best mlogloss: 0.9772133082300454   best auc: 0.9960571601879297   best loss: 0.3513079881668091
- 第4折中，best mlogloss: 0.9771865291289459   best auc: 0.9979561512414352   best loss: 0.3584481477737427
- 第5折中，best mlogloss: 0.9822103989262112   best auc: 0.9976901364500118   best loss: 0.31363099813461304
- 第6折中，best mlogloss: 0.9818852323759074   best auc: 0.9982848531202542   best loss: 0.3530382215976715
- 第7折中，best mlogloss: 0.9791612508478834   best auc: 0.9988892924512657   best loss: 0.33308154344558716
- 第8折中，best mlogloss: 0.98057134271314   best auc: 0.9992422836399381   best loss: 0.33398792147636414
- 第9折中，best mlogloss: 0.9789818534174326   best auc: 0.9985592272795051   best loss: 0.3301367461681366
- 第10折中，best mlogloss: 0.980113297001539   best auc: 0.9970231641648527   best loss: 0.3556895852088928

In [ ]:
# model_num = 11
model = NeuralNet(w2v_model.wv.vectors.shape[0]+1,w2v_model.wv.vectors.shape[1]+fasttext_model.wv.vectors.shape[1],embeddings=True)
model.to(DEVICE)
test_preds_total = []
test_dataloader, test_torchdata = get_dataloader(test_dataset, mode='test')
for i in range(1,model_num):
    model.load_state_dict(torch.load('{}_model_{}.bin'.format(NAME, i)))
    test_pred_results = validation_funtion(model, test_dataloader, test_torchdata, 'test')
    test_preds_total.append(test_pred_results)
test_preds_merge = np.sum(test_preds_total, axis=0) / (model_num-1)
pres_fold = [[str(p) for p in li] for li in test_preds_merge]
pres_all = [' '.join(p) for p in pres_fold]
str_w = ''
sub_id = test_data['report_ID'].values
with open('submit.csv','w') as f:
    for i in range(len(sub_id)):
        str_w += sub_id[i] + '|,' + '|' + pres_all[i] + '\n'
    str_w = str_w.strip('\n')
    f.write(str_w)